In [4]:
import pandas as pd
import numpy as np
import requests
import ast

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. 
El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.
- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [5]:
df_attacks = pd.read_csv('../datos/attacks_limpieza_completa.csv', index_col=0)
df_meteo = pd.read_csv('../datos/paises_meteo.csv', index_col=0)

In [6]:
df_attacks.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [7]:
df_attacks_paises = df_attacks[df_attacks['country'].isin(['australia', 'usa', 'new zealand', 'south africa', 'papua new guinea'])]

In [8]:
df_attacks_paises['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [9]:
df_meteo.sample(3)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
16,51,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",27,-1,5,1010,none,3,0,250,2,39.783730,-100.445882,USA
167,120,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 10, 'speed': ...",16,2,13,1017,rain,7,0,10,4,-28.816624,24.991639,SOUTH AFRICA
82,57,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 140, 'speed':...",21,6,9,1024,rain,3,0,140,3,-24.776109,134.755000,AUSTRALIA


In [10]:
df_meteo.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 235, 'speed':...",24,-1,11,1007,rain,2,0,290,2,39.783730,-100.445882,USA
1,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 275, 'speed':...",24,-4,10,1008,rain,3,0,300,2,39.783730,-100.445882,USA
2,2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",26,-4,7,1009,none,3,0,220,3,39.783730,-100.445882,USA
3,3,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",28,-1,3,1009,none,3,0,210,3,39.783730,-100.445882,USA
4,4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",29,-1,2,1008,none,3,0,185,3,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,180,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",25,2,12,1009,rain,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,316,183,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,2,11,1010,rain,3,0,85,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,317,186,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,2,10,1012,rain,3,0,90,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,318,189,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",26,6,10,1011,none,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [11]:
df_meteo.dtypes

timepoint              int64
cloudcover             int64
highcloud              int64
midcloud               int64
lowcloud               int64
rh_profile            object
wind_profile          object
temp2m                 int64
lifted_index           int64
rh2m                   int64
msl_pressure           int64
prec_type             object
prec_amount            int64
snow_depth             int64
wind10m.direction      int64
wind10m.speed          int64
latitud              float64
longitud             float64
pais                  object
dtype: object

In [12]:
df_meteo['wind_profile']= df_meteo['wind_profile'].apply(ast.literal_eval)

In [13]:
df_meteo['wind_profile']


0      [{'layer': '950mb', 'direction': 235, 'speed':...
1      [{'layer': '950mb', 'direction': 275, 'speed':...
2      [{'layer': '950mb', 'direction': 225, 'speed':...
3      [{'layer': '950mb', 'direction': 225, 'speed':...
4      [{'layer': '950mb', 'direction': 225, 'speed':...
                             ...                        
315    [{'layer': '950mb', 'direction': 80, 'speed': ...
316    [{'layer': '950mb', 'direction': 90, 'speed': ...
317    [{'layer': '950mb', 'direction': 90, 'speed': ...
318    [{'layer': '950mb', 'direction': 80, 'speed': ...
319    [{'layer': '950mb', 'direction': 80, 'speed': ...
Name: wind_profile, Length: 320, dtype: object

In [14]:
x = df_meteo['wind_profile'].apply(pd.Series)

In [15]:
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 235, 'speed': 2}","{'layer': '900mb', 'direction': 235, 'speed': 3}","{'layer': '850mb', 'direction': 235, 'speed': 4}","{'layer': '800mb', 'direction': 230, 'speed': 4}","{'layer': '750mb', 'direction': 215, 'speed': 5}","{'layer': '700mb', 'direction': 215, 'speed': 5}","{'layer': '650mb', 'direction': 230, 'speed': 6}","{'layer': '600mb', 'direction': 240, 'speed': 7}","{'layer': '550mb', 'direction': 240, 'speed': 8}","{'layer': '500mb', 'direction': 230, 'speed': 8}","{'layer': '450mb', 'direction': 215, 'speed': 8}","{'layer': '400mb', 'direction': 210, 'speed': 8}","{'layer': '350mb', 'direction': 210, 'speed': 8}","{'layer': '300mb', 'direction': 220, 'speed': 9}","{'layer': '250mb', 'direction': 220, 'speed': 9}","{'layer': '200mb', 'direction': 230, 'speed': 10}"
1,"{'layer': '950mb', 'direction': 275, 'speed': 3}","{'layer': '900mb', 'direction': 285, 'speed': 3}","{'layer': '850mb', 'direction': 245, 'speed': 4}","{'layer': '800mb', 'direction': 215, 'speed': 4}","{'layer': '750mb', 'direction': 210, 'speed': 5}","{'layer': '700mb', 'direction': 230, 'speed': 6}","{'layer': '650mb', 'direction': 235, 'speed': 6}","{'layer': '600mb', 'direction': 235, 'speed': 7}","{'layer': '550mb', 'direction': 235, 'speed': 7}","{'layer': '500mb', 'direction': 235, 'speed': 7}","{'layer': '450mb', 'direction': 225, 'speed': 7}","{'layer': '400mb', 'direction': 215, 'speed': 8}","{'layer': '350mb', 'direction': 215, 'speed': 8}","{'layer': '300mb', 'direction': 215, 'speed': 9}","{'layer': '250mb', 'direction': 215, 'speed': 9}","{'layer': '200mb', 'direction': 230, 'speed': 10}"
2,"{'layer': '950mb', 'direction': 225, 'speed': 3}","{'layer': '900mb', 'direction': 280, 'speed': 3}","{'layer': '850mb', 'direction': 265, 'speed': 4}","{'layer': '800mb', 'direction': 240, 'speed': 4}","{'layer': '750mb', 'direction': 230, 'speed': 5}","{'layer': '700mb', 'direction': 225, 'speed': 6}","{'layer': '650mb', 'direction': 230, 'speed': 6}","{'layer': '600mb', 'direction': 235, 'speed': 7}","{'layer': '550mb', 'direction': 235, 'speed': 7}","{'layer': '500mb', 'direction': 235, 'speed': 7}","{'layer': '450mb', 'direction': 235, 'speed': 7}","{'layer': '400mb', 'direction': 220, 'speed': 8}","{'layer': '350mb', 'direction': 215, 'speed': 9}","{'layer': '300mb', 'direction': 215, 'speed': 9}","{'layer': '250mb', 'direction': 215, 'speed': 10}","{'layer': '200mb', 'direction': 225, 'speed': 10}"
3,"{'layer': '950mb', 'direction': 225, 'speed': 3}","{'layer': '900mb', 'direction': 250, 'speed': 3}","{'layer': '850mb', 'direction': 265, 'speed': 4}","{'layer': '800mb', 'direction': 240, 'speed': 5}","{'layer': '750mb', 'direction': 225, 'speed': 5}","{'layer': '700mb', 'direction': 220, 'speed': 6}","{'layer': '650mb', 'direction': 225, 'speed': 6}","{'layer': '600mb', 'direction': 230, 'speed': 7}","{'layer': '550mb', 'direction': 230, 'speed': 8}","{'layer': '500mb', 'direction': 235, 'speed': 7}","{'layer': '450mb', 'direction': 225, 'speed': 7}","{'layer': '400mb', 'direction': 220, 'speed': 7}","{'layer': '350mb', 'direction': 215, 'speed': 8}","{'layer': '300mb', 'direction': 215, 'speed': 9}","{'layer': '250mb', 'direction': 215, 'speed': 10}","{'layer': '200mb', 'direction': 220, 'speed': 10}"
4,"{'layer': '950mb', 'direction': 225, 'speed': 4}","{'layer': '900mb', 'direction': 235, 'speed': 4}","{'layer': '850mb', 'direction': 240, 'speed': 4}","{'layer': '800mb', 'direction': 245, 'speed': 4}","{'layer': '750mb', 'direction': 240, 'speed': 5}","{'layer': '700mb', 'direction': 225, 'speed': 5}","{'layer': '650mb', 'direction': 230, 'speed': 6}","{'layer': '600mb', 'direction': 230, 'speed': 7}","{'layer': '550mb', 'direction': 225, 'speed': 7}","{'layer': '500mb', 'direction': 235, 'speed': 7}","{'layer': '450mb', 'direction': 235, 'speed': 7}","{'layer': '400mb', 'direction': 230, 'speed': 6}","{'layer': '350mb', 'direction': 230, 'speed': 7}","{'layer': '300mb'

In [16]:
nombre = 'windspeed_' + str(x[0].apply(pd.Series)['layer'][0] + '_direction_' + str(x[0].apply(pd.Series)['direction'][0]))
nombre

'windspeed_950mb_direction_235'

In [17]:
valores = list(x[0].apply(pd.Series)["speed"] )
valores

[2,
 3,
 3,
 3,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 4,
 3,
 2,
 2,
 2,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 5,
 5,
 5,
 5,
 6,
 5,
 2,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 5,
 5,
 6,
 5,
 4,
 4,
 5,
 4,
 3,
 5,
 5,
 5,
 4,
 5,
 4,
 4,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 5,
 5,
 5,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [18]:
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = 'windspeed_' + str(x[i].apply(pd.Series)['layer'][0])

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["speed"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_meteo.insert(i, nombre, valores)


In [19]:
df_meteo

,windspeed_950mb,windspeed_900mb,windspeed_850mb,windspeed_800mb,windspeed_750mb,windspeed_700mb,windspeed_650mb,windspeed_600mb,windspeed_550mb,windspeed_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,2,3,4,4,5,5,6,7,8,8,...,11,1007,rain,2,0,290,2,39.783730,-100.445882,USA
1,3,3,4,4,5,6,6,7,7,7,...,10,1008,rain,3,0,300,2,39.783730,-100.445882,USA
2,3,3,4,4,5,6,6,7,7,7,...,7,1009,none,3,0,220,3,39.783730,-100.445882,USA
3,3,3,4,5,5,6,6,7,8,7,...,3,1009,none,3,0,210,3,39.783730,-100.445882,USA
4,4,4,4,4,5,5,6,7,7,7,...,2,1008,none,3,0,185,3,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,4,5,5,4,4,3,3,3,3,3,...,12,1009,rain,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,5,5,4,4,3,3,3,3,4,3,...,11,1010,rain,3,0,85,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,5,5,5,4,3,3,3,3,4,3,...,10,1012,rain,3,0,90,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,5,5,5,4,3,3,3,3,4,3,...,10,1011,none,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [20]:
df_meteo['rh_profile']= df_meteo['rh_profile'].apply(ast.literal_eval)

In [21]:
x = df_meteo['rh_profile'].apply(pd.Series)

In [22]:
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = 'rh_' + str(x[i].apply(pd.Series)['layer'][0] )
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_meteo.insert(i, nombre, valores)

In [23]:
df_meteo

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,12,10,10,10,10,9,6,5,10,13,...,11,1007,rain,2,0,290,2,39.783730,-100.445882,USA
1,8,7,6,9,8,3,1,3,7,13,...,10,1008,rain,3,0,300,2,39.783730,-100.445882,USA
2,6,4,4,7,5,1,0,6,11,7,...,7,1009,none,3,0,220,3,39.783730,-100.445882,USA
3,4,6,6,4,3,4,6,8,10,5,...,3,1009,none,3,0,210,3,39.783730,-100.445882,USA
4,2,3,5,8,8,7,3,3,8,4,...,2,1008,none,3,0,185,3,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,15,12,12,11,2,-3,-4,-4,-4,-4,...,12,1009,rain,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,14,12,10,8,0,-3,-4,-4,-4,-4,...,11,1010,rain,3,0,85,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,13,14,6,4,-3,-4,-4,-4,-4,-4,...,10,1012,rain,3,0,90,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,12,14,1,-1,-3,-3,-4,-4,-4,-4,...,10,1011,none,3,0,80,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [24]:
df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())

/tmp/ipykernel_15317/2787290623.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())


In [25]:
df_meteo_medias.reset_index(inplace=True)

In [26]:
df_meteo_medias['pais'] = df_meteo_medias['pais'].apply(lambda x: x.lower())

In [27]:
df_meteo_medias

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,australia,12.765625,10.906250,5.890625,1.328125,0.406250,0.203125,-0.296875,-1.015625,-1.515625,...,21.328125,4.765625,10.171875,1022.859375,2.984375,0.0,114.843750,3.140625,-24.776109,134.755000
1,new zealand,11.296875,13.375000,9.546875,3.687500,0.578125,-0.640625,-0.500000,-0.203125,-0.484375,...,9.843750,9.468750,8.437500,1025.203125,2.593750,0.0,178.906250,3.046875,-41.500083,172.834408
2,papua new guinea,13.593750,11.890625,9.156250,6.093750,1.062500,-0.875000,-2.218750,-3.218750,-3.625000,...,25.781250,1.609375,10.953125,1011.796875,2.281250,0.0,105.390625,3.703125,-5.681607,144.248908
3,south africa,12.921875,11.984375,11.312500,8.890625,8.515625,8.953125,8.875000,8.421875,7.609375,...,16.609375,3.937500,12.484375,1018.046875,5.546875,0.0,163.125000,3.203125,-28.816624,24.991639
4,usa,8.546875,9.578125,9.265625,9.015625,8.078125,5.859375,3.656250,2.906250,3.234375,...,25.140625,0.140625,6.437500,1012.187500,2.984375,0.0,175.390625,2.296875,39.783730,-100.445882


In [28]:
df_final = pd.merge(left = df_attacks_paises, right= df_meteo_medias, how= "left", left_on= "country", right_on= "pais")
df_final.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,longitud_x,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_y,longitud_y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,25.140625,0.140625,6.4375,1012.1875,2.984375,0.0,175.390625,2.296875,39.78373,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,25.140625,0.140625,6.4375,1012.1875,2.984375,0.0,175.390625,2.296875,39.78373,-100.445882


In [30]:
df_final.to_csv('../datos/attacks_meteo.csv')